# Sentiment Embeddings Model Evaluation

This notebook loads the archived sentiment model and evaluates it on the test set. Metrics and visualizations are saved to the evaluation directory.

In [ ]:
# Setup: imports and paths
import sys
from pathlib import Path
import json
import numpy as np
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
import matplotlib.pyplot as plt

EVAL_DIR = Path('../evaluation/sentiment_embeddings').resolve()
MODEL_DIR = Path('../../models/sentiment_embeddings/sentiment_distilbert_best/sentiment_distilbert/best').resolve()
SPLITS_PATH = Path('../../outputs/sentiment_embeddings/preprocessing/splits.json').resolve()

sys.path.insert(0, str(EVAL_DIR))
from eval_utils import compute_metrics

In [ ]:
# Load test split and labels
with open(SPLITS_PATH, 'r', encoding='utf-8') as f:
    splits = json.load(f)
labels = splits['labels']
test_records = splits['splits']['test']

test_df = pd.DataFrame([{'text': r['text'], 'label': labels.index(r['label'])} for r in test_records])

In [ ]:
# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model.eval()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

In [ ]:
# Run inference
all_preds, all_labels = [], []
with torch.no_grad():
    for _, row in test_df.iterrows():
        inputs = tokenizer(row['text'], return_tensors='pt', truncation=True, padding=True).to(device)
        logits = model(**inputs).logits.cpu().numpy()[0]
        pred = np.argmax(logits)
        all_preds.append(pred)
        all_labels.append(row['label'])
all_preds, all_labels = np.array(all_preds), np.array(all_labels)

In [ ]:
# Compute and save metrics
metrics = compute_metrics(all_labels, all_preds, labels)
with open(EVAL_DIR / 'metrics.json', 'w', encoding='utf-8') as f:
    json.dump(metrics, f, indent=2)
metrics

In [ ]:
# Visualize confusion matrix
cm = np.array(metrics['confusion_matrix'])
fig, ax = plt.subplots(figsize=(8,8))
im = ax.imshow(cm, cmap='Blues')
ax.set_xticks(np.arange(len(labels)))
ax.set_yticks(np.arange(len(labels)))
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.set_yticklabels(labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.colorbar(im)
plt.tight_layout()
plt.savefig(EVAL_DIR / 'confusion_matrix.png')
plt.show()

## Results
- Metrics are saved to `metrics.json` in the evaluation directory.
- Confusion matrix is saved as `confusion_matrix.png`.
- See the classification report in the metrics output above.